## 回调函数常用内容
模型检查点（model checkpointing）：在训练过程中的不同时间点保存模型的当前权重。
提前终止（early stopping）：如果验证损失不再改善，则中断训练（当然，同时保存在训练过程中得到的最佳模型）。
在训练过程中动态调节某些参数值：比如优化器的学习率。
在训练过程中记录训练指标和验证指标，或将模型学到的表示可视化

## ModelCheckpoint、EarlyStopping 和 ReduceLROnPlateau。

In [ ]:
from tensorflow import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
        # 监控精度
        monitor='acc',
        # 如果在多于1轮没有提升，则中断训练
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        # 检查点文件
        filepath='my_model.h5',
        # 监控验证损失
        monitor='val_loss',
        # 只保存最佳模型
        save_best_only=True,
    )
]
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
# 在fit时加入callback_list
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          # 因为监控验证损失，所以需要验证集
          validation_data=(x_val, y_val))

callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        # 触发时学习率*factor
        factor=0.1,
        # 十轮内没有提升，则触发回调函数
        patience=10,
    )
]
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

## 也可以自己定义回调函数，需要了上网搜

## TensorBoard 以一维CNN为例，先编译一个模型

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 2000
max_len = 500
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________

## 使用TensorBoard回调函数训练模型

In [5]:
import os
if "my_log_dir" not in os.listdir():
    os.mkdir("my_log_dir")

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
        # 几轮之后记录一次直方图
        histogram_freq=1,
        # 几轮之后记录嵌入数据
        embeddings_freq=1,
    )
]
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

2022-04-26 20:51:40.342564: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-04-26 20:51:40.342743: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


Epoch 1/20
  5/157 [..............................] - ETA: 5s - loss: 7.8571 - acc: 0.4906

2022-04-26 20:51:41.123655: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-04-26 20:51:41.123790: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-04-26 20:51:41.207825: I tensorflow/core/profiler/internal/gpu/device_tracer.cc:223]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-04-26 20:51:41.213884: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating directory: my_log_dir/train/plugins/profile/2022_04_26_20_51_41
2022-04-26 20:51:41.214726: I tensorflow/core/profiler/rpc/client/save_profile.cc:182] Dumped gzipped tool data for trace.json.gz to my_log_dir/train/plugins/profile/2022_04_26_20_51_41/cfn-super-server.trace.json.gz
2022-04-26 20:51:41.224279: I tensorflow/core/profiler/rpc/client/save_profile.cc:176] Creating di

157/157 [==============================] - 4s 26ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 2/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 3/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 4/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 5/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 6/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 7/20
157/157 [==============================] - 4s 24ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 8/20
157/157 [==============================] - 4s 24ms/step - loss: 7.736

## keras.utils.plot_model 函数，它可以将模型绘制为层组成的图，需要了上网搜